In [88]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()
import base64
import json
import numpy as np
import pandas as pd
from snowflake.connector.pandas_tools import write_pandas
import utils.sf_conn as sf_conn

In [89]:
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [90]:
# Encode Client ID and Client Secret
client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

auth_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f"Basic {client_creds_b64}"
}

payload = {
    "grant_type": "client_credentials"
}

auth_response = requests.post(auth_url, headers=headers, data=payload)
auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']

# Step 2: Make API Request with the Access Token
api_headers = {
    "Authorization": "Bearer " + access_token
}

# base_url = 'https://api.spotify.com/v1/'
# track_id = '6y0igZArWVi6Iz0rj35c1Y'
# r = requests.get(base_url + 'audio-features/' + track_id, headers=api_headers)
# r = r.json()
# r


In [4]:
auth_response_data

{'access_token': 'BQDVZ38L4i8_kjn3F6XGwCBq6JIgb52Bi9cpP4FcMzFBAMEu0anBD5FTxAV8WXHLr-8RWi6ThIUmhoa1_VI2UTh4NQKz_XWvuL4FuxNjA5Gxme8Kjgg',
 'token_type': 'Bearer',
 'expires_in': 3600}

In [ ]:
# def get_playlists_name(access_token, playlist_id):
#     url = f"https://api.spotify.com/v1/playlists/{playlist_id}"
#     headers = {'Authorization': f'Bearer {access_token}'}
#     response = requests.get(url, headers=headers)
#     playlist_name = response.json().get("name")


In [ ]:
# url = f"https://api.spotify.com/v1/tracks/6dLLEE96JRnX1ngkL77yiA"
# headers = {'Authorization': f'Bearer {access_token}'}
# response = requests.get(url, headers=headers)
# track_data = response.json()
# artist_popularity = []
# # if "artists" in track_data:
# for artist in track_data["artists"]:
#     print(artist)
#     popularity = artist.get("type")
#     name = artist.get("name", "Unknown Artist")
#     artist_popularity.append((name, popularity))

# # Output the result
# for name, popularity in artist_popularity:
#     print(f"Artist: {name}, Popularity: {popularity}")

In [ ]:
# def get_artist_genres(access_token, artist_id):
#     url = f"https://api.spotify.com/v1/artists/{artist_id}"
#     headers = {'Authorization': f'Bearer {access_token}'}
#     response = requests.get(url, headers=headers)

#     if response.status_code == 200:
#         artist_data = response.json()
#         print(artist_data)
#         return ', '.join(artist_data.get('genres', []))
#     else:
#         print(f"Error fetching genres for artist {artist_id}")
#         return ''

In [104]:
def get_artist_genres(access_token, artist_id):
        url = f"https://api.spotify.com/v1/artists/{artist_id}"
        headers = {'Authorization': f'Bearer {access_token}'}
        response = requests.get(url, headers=headers)
        print(response.status_code)
        print(response.headers)

        if response.status_code == 200:
            artist_data = response.json()
            return ', '.join(artist_data.get('genres', []))
        else:
            print(f"Error fetching genres for artist {artist_id}")
            return ''

In [105]:
import requests
import pandas as pd
from json.decoder import JSONDecodeError

def get_playlist_name_tracks(access_token, playlist_ids):
    all_playlists_data = []
    for ids in playlist_ids:
        # Get the playlist's name
        url = f"https://api.spotify.com/v1/playlists/{ids}"
        headers = {'Authorization': f'Bearer {access_token}'}
        name_response = requests.get(url, headers=headers)
        # print(f"Fetching playlist details for ID: {ids}")
        # print(f"Request URL: {url}")
        # print(f"Status Code: {name_response.status_code}")
        # print(f"Response Content: {name_response.text}")

        # Check the response status and decode JSON
        if name_response.status_code != 200:
            print(f"Failed to fetch playlist name for {ids}: {name_response.status_code}")
            continue
        try:
            playlist_name = name_response.json().get("name", "Unknown Playlist")
        except JSONDecodeError:
            print(f"JSON decoding failed for playlist name {ids}")
            continue

        # Initialize pagination variables
        limit = 50
        offset = 0
        total_tracks = None

        # Loop through the paginated endpoint until all tracks have been fetched
        while total_tracks is None or offset < total_tracks:
            # Get the playlist's tracks
            tracks_url = f'https://api.spotify.com/v1/playlists/{ids}/tracks?limit={limit}&offset={offset}'
            track_response = requests.get(tracks_url, headers=headers)
  

            
            if track_response.status_code != 200:
                print(f"Failed to fetch tracks for playlist {ids}: {track_response.status_code}")
                break
            try:
                track_data = track_response.json()

                #print(track_data)
            except JSONDecodeError:
                print(f"JSON decoding failed for tracks in playlist {ids}")
                break

            # Update the total tracks count (only once)
            if total_tracks is None:
                total_tracks = track_data.get('total', 0)
            
            # Process each track
            for item in track_data.get('tracks', {}).get('items', []):
                track = item.get('track')
                if track:
                    track_id = track.get('id')
                    track_name = track.get('name')
                    artist_names = ', '.join(artist.get('name', '') for artist in track.get('artists', []))
                    artist_ids = ', '.join(artist.get('id', '') for artist in track.get('artists', []))
                    genres_list = [get_artist_genres(access_token, artist_id) for artist_id in artist_ids.split(', ')]
                    genres = '; '.join(genres_list)

                    all_playlists_data.append({
                        'Playlist_Id': ids,
                        "Playlist_Name": playlist_name,
                        'Track_Name': track_name,
                        "Artist_Names": artist_names,
                        "Artist_Id": artist_ids,
                        "Track_Id": track_id,
                        "Genres": genres
                    })
                
            # Increment the offset for the next page of results
            offset += limit

    # Create a DataFrame from the collected data
        playlist_df = pd.DataFrame(all_playlists_data)
    return playlist_df



In [96]:
def get_playlist_name_tracks_2(access_token, playlist_ids):
    all_playlists_data = []
    for ids in playlist_ids:
        # Get the playlist's name
        url = f"https://api.spotify.com/v1/playlists/{ids}"
        headers = {'Authorization': f'Bearer {access_token}'}
        name_response = requests.get(url, headers=headers)

        if name_response.status_code != 200:
            print(f"Failed to fetch playlist name for {ids}: {name_response.status_code}")
            continue
        try:
            playlist_name = name_response.json().get("name", "Unknown Playlist")
        except JSONDecodeError:
            print(f"JSON decoding failed for playlist name {ids}")
            continue

        # Get the playlist's tracks (up to 50 tracks)
        tracks_url = f'https://api.spotify.com/v1/playlists/{ids}/tracks?limit=50'
        track_response = requests.get(tracks_url, headers=headers)
        print(track_response)

        if track_response.status_code != 200:
            print(f"Failed to fetch tracks for playlist {ids}: {track_response.status_code}")
            continue
        try:
            track_data = track_response.json()
            print(track_data)
        except JSONDecodeError:
            print(f"JSON decoding failed for tracks in playlist {ids}")
            continue

        # Process each track
        for item in track_data.get('tracks', {}).get('items', []):
            track = item.get('track')
            if track:
                track_id = track.get('id')
                track_name = track.get('name')
                artist_names = ', '.join(artist.get('name', '') for artist in track.get('artists', []))
                artist_ids = ', '.join(artist.get('id', '') for artist in track.get('artists', []))
               # genres_list = [get_artist_genres(access_token, artist_id) for artist_id in artist_ids.split(', ')]
               # genres = '; '.join(genres_list)

                all_playlists_data.append({
                    'Playlist_Id': ids,
                    "Playlist_Name": playlist_name,
                    'Track_Name': track_name,
                    "Artist_Names": artist_names,
                    "Artist_Id": artist_ids,
                    "Track_Id": track_id
                  #  "Genres": genres
                })

    # Create a DataFrame from the collected data
    playlist_df = pd.DataFrame(all_playlists_data)
    return playlist_df

In [106]:
test_df = get_playlist_name_tracks(access_token = access_token, playlist_ids=["37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c"])
test_df


200
{'content-type': 'application/json; charset=utf-8', 'cache-control': 'public, max-age=7200', 'x-robots-tag': 'noindex, nofollow', 'access-control-allow-origin': '*', 'access-control-allow-headers': 'Accept, App-Platform, Authorization, Content-Type, Origin, Retry-After, Spotify-App-Version, X-Cloud-Trace-Context, client-token, content-access-token', 'access-control-allow-methods': 'GET, POST, OPTIONS, PUT, DELETE, PATCH', 'access-control-allow-credentials': 'true', 'access-control-max-age': '604800', 'content-encoding': 'gzip', 'strict-transport-security': 'max-age=31536000', 'x-content-type-options': 'nosniff', 'date': 'Sat, 27 Jan 2024 22:17:54 GMT', 'server': 'envoy', 'Via': 'HTTP/2 edgeproxy, 1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'Transfer-Encoding': 'chunked'}
200
{'content-type': 'application/json; charset=utf-8', 'cache-control': 'public, max-age=7200', 'x-robots-tag': 'noindex, nofollow', 'access-control-allow-origin': '*', 'access-contro

,Playlist_Id,Playlist_Name,Track_Name,Artist_Names,Artist_Id,Track_Id,Genres
0,37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c,Dance/Electronic Mix,Lean On (feat. MØ & DJ Snake),"Major Lazer, MØ, DJ Snake","738wLrAtLtCtFOLvQBXOXp, 0bdfiayQAKewqEvaU6rXCv...",1qE47wUKG2juJwPoLqg4C9,"dance pop, edm, electro house, moombahton, pop..."
1,37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c,Dance/Electronic Mix,On My Love,"Zara Larsson, David Guetta","1Xylc3o4UrD53lo9CvFvVg, 1Cs0zKBU1kc0i8ypK3B9ai",0K6iKgHPPhAb4Acmg9aD77,"pop, scandipop, swedish electropop, swedish po..."
2,37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c,Dance/Electronic Mix,Feel So Close - Radio Edit,Calvin Harris,7CajNmpbOovFoOoasH2HaY,1gihuPhrLraKYrJMAEONyc,"dance pop, edm, electro house, house, pop, pop..."
3,37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c,Dance/Electronic Mix,Better Off Alone,Alice Deejay,2tbvDi9eXf9XXp06LupkED,5XVjNRubJUW0iPhhSWpLCj,eurodance
4,37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c,Dance/Electronic Mix,I'm Good (Blue),"David Guetta, Bebe Rexha","1Cs0zKBU1kc0i8ypK3B9ai, 64M6ah0SkkRsnPGtGiRAbb",4uUG5RXrOk84mYEfFvj3cK,"big room, dance pop, edm, pop, pop dance; danc..."
5,37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c,Dance/Electronic Mix,L'Amour Toujours,Gigi D'Agostino,1OAjDaKgg00KCUYqDe68un,52LJ3hyknOijCrE5gCD0rE,"eurodance, italo dance"
6,37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c,Dance/Electronic Mix,Blue (Da Ba Dee) - Gabry Ponte Ice Pop Radio,"Eiffel 65, Gabry Ponte","64rxQRJsLgZwHHyWKB8fiF, 5ENS85nZShljwNgg4wFD7D",2yAVzRiEQooPEJ9SYx11L3,"eurodance, europop, italian adult pop, italo d..."
7,37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c,Dance/Electronic Mix,Children,Robert Miles,2YVF0Ou5zIc4mpgtLIlGN0,4wtR6HB3XekEengMX17cpc,"dream trance, eurodance, trance"
8,37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c,Dance/Electronic Mix,One More Time,Daft Punk,4tZwfgrHOc3mvqYlEYSvVi,0DiWol3AO6WpXZgp0goxAV,"electro, filter house, rock"
9,37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c,Dance/Electronic Mix,Played-A-Live (The Bongo Song),Safri Duo,2UOx6w3eHpPKc3RBnNV3Rl,7uuo02BIR76qZ6ZXQhz7Ys,eurodance


In [10]:
def get_track_information(access_token, track_id):
    track_data_list = []
    # for id in track_id:
    url = f"https://api.spotify.com/v1/tracks/{track_id}"
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)
    print(response.headers['access-control-allow-headers'])
    # Check if the response status code indicates success
    if response.status_code == 200:
        try:
            track_data = response.json()
            #print(track_data)
            track_name = track_data.get('name')
            track_popularity = track_data.get('popularity')
            track_uri = track_data.get('uri')

            track_data_list.append({
                
            "Track_Id": track_id,
            "Track_Name": track_name,
            "Track_Popularity": track_popularity,
            "Track_Uri": track_uri
        })

        except:
            print("Error: Could not decode JSON. Response content:", response.text)
            raise
    else:
        # The API request was not successful
        print(f"Error: Received response with status code {response.status_code}. Response content:", response.text)
        return None
    return pd.DataFrame(track_data_list)


In [11]:
track_info = get_track_information(access_token= access_token, track_id = '6dLLEE96JRnX1ngkL77yiA')

track_info

Accept, App-Platform, Authorization, Content-Type, Origin, Retry-After, Spotify-App-Version, X-Cloud-Trace-Context, client-token, content-access-token


,Track_Id,Track_Name,Track_Popularity,Track_Uri
0,6dLLEE96JRnX1ngkL77yiA,"Boom (feat. MOTi, Ty Dolla $ign, Wizkid & Kran...",31,spotify:track:6dLLEE96JRnX1ngkL77yiA


In [ ]:
response.headers

In [ ]:
track_info.info()

In [ ]:
# def get_audio_features(access_token, track_ids):
def get_track_audio_features(access_token, track_ids):
    track_audio_features = []
    for id in track_ids:
        url = f"https://api.spotify.com/v1/audio-features/{id}"
        headers = {'Authorization': f'Bearer {access_token}'}
        response = requests.get(url, headers=headers)
        track = response.json()
        if response.status_code == 200:
            try:
                # for audio in repsone_json['track']:
                audio_data = {
                    'Track_id': id,
                    'danceability': track['danceability'],
                    'duration_ms': track['duration_ms'],
                    'energy': track['energy'],
                    'instrumentalness': track['instrumentalness'],
                    'key': track['key'],
                    'liveness': track['liveness'],
                    'loudness': track['loudness'],
                    'mode': track['mode'],
                    'speechiness': track['speechiness'],
                    'time_signature':track['time_signature'],
                    'track_uri': track['uri']

                }
                track_audio_features.append(audio_data)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON for workbook ID {id}: {e}")
        else:
            raise Exception(f"Error in API request: {response.status_code}")
    track_audio_features_df = pd.DataFrame(track_audio_features)
    return track_audio_features_df

In [ ]:
playlist_data = get_playlist_name_tracks(access_token=access_token, playlist_id=['37i9dQZF1EQp9BVPsNVof1?si=b0e0182f48ea461c'])
                                                                            # "37i9dQZF1EIh5VHows1LoY?si=3f242c7e9e7a43f8",
                                                                            # '37i9dQZF1EVKuMoAJjoTIw?si=2ed6df3d293b4519',
                                                                            # '37i9dQZF1EQpoj8u9Hn81e?si=92566118c4954e73',
                                                                            # '37i9dQZF1EQqlvxWrOgFZm?si=2b9ab93cc7714708',
                                                                            # '37i9dQZF1EIhBF9gUW4UsT?si=089940507c6b46f3',
                                                                            # '37i9dQZF1EIherXksVvnrN?si=d8e28d16e28b40b0',
                                                                            # '37i9dQZF1EVHGWrwldPRtj?si=98356d3db1994d67',
                                                                            # "37i9dQZF1DWSf2RDTDayIx?si=1d3f916447094372"
                                                                            # "37i9dQZF1DX0AMssoUKCz7?si=80092935821a4201",
                                                                            # '37i9dQZF1DX0hvSv9Rf41p?si=72b7f2562f0a4020',
                                                                            # "6FSFUWzuF2KigQDKV42Uru?si=cfc88421610b46c3",
                                                                            # '37i9dQZF1E35DbZEmy06Qp?si=b6e38bde3d8d4116'])

In [ ]:
playlist_data.head()

In [ ]:
audio_features_df = get_track_audio_features(access_token = access_token, track_ids = playlist_data['Track_Id'])


In [ ]:
audio_features_df.head()